# Отчёт о результатах сканирования
![](_static/cmpl-reporting.png)

## Как именно будет происходить генерация pdf отчёта
![](_static/reporting-data-to-pdf.png)

## jinja2
### Основная идея шаблонизации

In [31]:
'Hello, {name}'.format(name='mike') 


'Hello, mike'

Форматные строки это хрошо, с помощью них можно параметризировать например регулярные выражения или html.

In [33]:
html_tpl = '''
<!doctype html>
<html>
    <head>
        <meta charset="utf-8"/>
        <title>{name}'s Homepage</title>
    </head>
    <body>
        <b><i>Hello, {name} </i></b>
    </div>
    </body>
</html>'''

In [34]:
from IPython.core.display import display, HTML
display(HTML(html_tpl.format(name='Mike')))

In [36]:
html_tpl = '''
<!doctype html>
<html>
    <head>
        <meta charset="utf-8"/>
        <meta name={{ name }}>
        <style type="text/css">
        .my_style {
        }
        </style>
        <title>{name}'s Homepage</title>
    </head>
    <body>
    <pre class="my_style">Hello, {{ name }}</pre>
    </body>
</html>'''

In [37]:
display(HTML(html_tpl.format(name='Mike')))

KeyError: '\n        '

In [38]:
from jinja2 import Environment, BaseLoader

rtemplate = Environment(loader=BaseLoader).from_string(html_tpl)
rendered_html = rtemplate.render(name='Mike')

display(HTML(rendered_html))

Хочется еще много всякого, загружать из внешнегофайла, чтобы редактирвоание шаблонов было независимым от программирования.
Циклы, наследование, включение блоков.

http://jinja.pocoo.org/docs/2.10/

In [39]:
!pip install jinja2

Cleaning up...


### Инициализация

In [8]:
from jinja2 import Environment, PackageLoader, FileSystemLoader, select_autoescape
env = Environment(
    loader=FileSystemLoader('templates'),
    autoescape=select_autoescape(['html', 'xml'])
)

In [9]:
# autoescape - автоматическое экранирование специальных символов в html 
# loader - загрузчик, шаблон можно получить разными способами из строки (см. выше) или из файла

### Переменные
`{{ param }}` - так обозначается подстановка переменных. 

### Загрузка шаблона и рендеринг

In [41]:
test_tpl = env.get_template('test.html')
test_tpl

<Template 'test.html'>

In [42]:
rendered_html = test_tpl.render(name='Mike')
print(rendered_html)


<!doctype html>
<html>
    <head>
        <meta charset="utf-8"/>
        <style type="text/css">
        .my_style {
            
        }
        </style>
        <title>Mike's Homepage</title>
    </head>
    <body>
        Mike 
    </body>
</html>


In [43]:
display(HTML(rendered_html))

### Коментарии
`{# This would take no effect #}`

### Доступ по ключам словаря - через точку
`{{ dict_.key }}` - так обозначается подстановка переменных. 

In [44]:
user_data = dict(
    name = "Mike",
    email = "mike@mike.com"
)

In [45]:
test_tpl = env.get_template('test.html')
rendered_html = test_tpl.render(user_data=user_data)
print(rendered_html)

<!doctype html>
<html>
    <head>
        <meta charset="utf-8"/>
        <style type="text/css">
        .my_style {
            
        }
        </style>
        <title>Mike's Homepage</title>
    </head>
    <body>
        <p> name:Mike </p>
        <p> email:mike@mike.com </p>
    </body>
</html>


Кстати

In [46]:
'This is {user_data[name]} homepage! Email me at {user_data[email]}'.format(user_data=user_data)

'This is Mike homepage! Email me at mike@mike.com'

###  Циклы
{% for user in users %}
  <li><a href="{{ user.url }}">{{ user.username }}</a></li>
{% endfor %}

In [47]:
import hashlib
hashlib.sha1('ff'.encode('utf8')).hexdigest()

'ed70c57d7564e994e7d5f6fd6967cea8b347efbc'

In [48]:
user_list = ['user {}'.format(n) for n in range(0, 10)]
pwd_hash_list = [hashlib.sha1(username.encode('utf8')).hexdigest() for username in user_list]

In [49]:
from collections import namedtuple
User = namedtuple('User','name, pwd_hash')

In [50]:
users = [User(name,pwd_hash) for name, pwd_hash in zip(user_list,pwd_hash_list)]
users

[User(name='user 0', pwd_hash='0e029f365a9139f037a6e54076e645b4b04d741a'),
 User(name='user 1', pwd_hash='40c4781874d18976c889831b1d061bdce211df40'),
 User(name='user 2', pwd_hash='be0be2895fa4913a698c240b63117745b153c121'),
 User(name='user 3', pwd_hash='4eb9a6f82b1b8156155e59b37fd84f2f5e03fc87'),
 User(name='user 4', pwd_hash='4d5fffe1a530e89f27cebde5c491af2c4d420fef'),
 User(name='user 5', pwd_hash='393748d27a163a49e4a171c26e0793082ab398d2'),
 User(name='user 6', pwd_hash='f05e3d430b3f430e6f0b9c2002aed2c9dd0b090e'),
 User(name='user 7', pwd_hash='d77a6955a087b54dcb5c48c0c1588cfe246be1a7'),
 User(name='user 8', pwd_hash='609013053fa9dd58df8584836c651182235bdc65'),
 User(name='user 9', pwd_hash='ba932c9e70b22c88508565ed1888c95205d23b39')]

In [51]:
test_tpl = env.get_template('test.html')
rendered_html = test_tpl.render(user_data=user_data,users=users)
print(rendered_html)

<!doctype html>
<html>
    <head>
        <meta charset="utf-8"/>
        <style type="text/css">
        .my_style {
            
        }
        </style>
        <title>Mike's Homepage</title>
    </head>
    <body>
        <p> name:Mike </p>
        <p> email:mike@mike.com </p>
         <article>
             
             <div>
                 <p> User name: user 0 </p>
                 <p> Password hash: 0e029f365a9139f037a6e54076e645b4b04d741a </p>
             </div>
             
             <div>
                 <p> User name: user 1 </p>
                 <p> Password hash: 40c4781874d18976c889831b1d061bdce211df40 </p>
             </div>
             
             <div>
                 <p> User name: user 2 </p>
                 <p> Password hash: be0be2895fa4913a698c240b63117745b153c121 </p>
             </div>
             
             <div>
                 <p> User name: user 3 </p>
                 <p> Password hash: 4eb9a6f82b1b8156155e59b37fd84f2f5e03fc87 </p>
 

In [52]:
display(HTML(rendered_html))

### Weasyprint
http://weasyprint.readthedocs.io/en/latest/tutorial.html#as-a-python-library

In [53]:
!pip install weasyprint

Cleaning up...


In [54]:
from weasyprint import HTML
HTML(url='http://weasyprint.org').write_pdf('./weasyprint.org.pdf')

Можно генерировать pdf или png файлы из 
    * Сайтов
    * Html файлов
    * Строк с html 

In [55]:
document = HTML(string=rendered_html).render()
document

In [56]:
len(document.pages)

1

In [57]:
document.pages[0].links

[]

In [58]:
document.metadata.title

"Mike's Homepage"

In [59]:
document.write_pdf('mikes.pdf')

### Шаблоны для отчётов

In [60]:
from weasyprint import HTML, CSS
template = env.get_template('index.html')

whtml = HTML(string=template.render().encode('utf8'))

wcss = CSS(filename='./templates/style.css')

whtml.write_pdf('sample_report.pdf',stylesheets=[wcss])

### Задача
В модуле reporting.py реализовать функционал создания pdf отчета. Можно пользоваться как выше предложеными технологиями, так и любыми другими.
Фунционал должен генеририровать pdf отчёт с использованием данных из таблицы scandata и фала env.json.
* Информация о сканировании - дата, продолжительность, общее количество проверок, количество проверок по статусам.
* Информация о системе - какие транспорты использовались, данные о подключении
* Информация о каждом выполненом контроле - заголовок, описание, требование, статус (совместимо, несовместимо, ошибка)  
В таблицы controls и scandata добавить соответвующие поля. 

Если отчет неполучится сделать именно в формате pdf. (неполучится поставить weasyprint и эксперементы с другими библиотеками не увенчаются успехом - отчет в формате html - ok)